## **Data Science Workplow**

Berliner Hochschule für Technik

News Reader project - Cat news

Polina Kozyr

In this project I extracted news about cats from cat news webpage and made topic classification and sentiment analysis of news. For topic classification I trained classificator to predict topic of cat news. For sentiment analysis I just used already pre-trained model to classify news on positive and negative ones. Also I made a web-application to make an end-to-end solution for the cat news reader.

Project content:

1. Extraction of news from website - creation of cat news dataset
2. Topic classification
        1) Cleaning of news texts
        2) Formation of topic classification system stages
        2) Classification for humorous or not humorous articles
        3) Classification for other categories
        4) Сombining results from two classifications
3. Sentiment analysis
4. Web-application creation


## 1. Extraction of news from website - creation of cat news dataset

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

import string
import re
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pickle

### Get title and link of news from pages that have list of news

In [94]:
news_data = []
for subpage_index in range(1, 200):
    page = "http://catdailynews.com/page/" + str(subpage_index) + "/"
    print(page)
    protocol_list = requests.get(page).text
    soup = BeautifulSoup(protocol_list, 'lxml')
    
    for ai in soup.find_all('a'):
        if ai.div:
            title = ai.div.text
            link = ai.get('href')
            print(title)
            print(link)
            news_data.append((title, link))


http://catdailynews.com/page/1/
Indiana Jones and a Cat
http://catdailynews.com/2023/06/indiana-jones-and-a-cat/
Interesting Cat Facts
http://catdailynews.com/2023/06/interesting-cat-facts/
Cat Gets “Jailed” For Being Too Daring
http://catdailynews.com/2023/06/cat-gets-jailed-for-being-too-daring/
Toys to Keep Your Cat Happy
http://catdailynews.com/2023/06/toys-to-keep-your-cat-happy/
The Evolution of Cats
http://catdailynews.com/2023/06/the-evolution-of-cats/
Video Games Where You Play a Cat
http://catdailynews.com/2023/06/video-games-where-you-play-a-cat/
University of Bristol Recognizing Catgender
http://catdailynews.com/2023/06/university-of-bristol-recognizing-catgender/
Cat Addicted to Mice Videos
http://catdailynews.com/2023/06/cat-addicted-to-mice-videos/
Farm for Feral Cats
http://catdailynews.com/2023/06/farm-for-feral-cats/
How to Break Up a Cat Fight
http://catdailynews.com/2023/06/how-to-break-up-a-cat-fight/
http://catdailynews.com/page/2/
Man Tracks His Cat with GPS
http

Adopt a Cartel Leader’s Tattooed Cat
http://catdailynews.com/2023/02/adopt-a-cartel-leaders-tattooed-cat/
Two Cats Interrupt Weatherman
http://catdailynews.com/2023/02/two-cats-interrupt-weatherman/
Fat Cat Becomes Polish Tourist Attraction
http://catdailynews.com/2023/02/fat-cat-becomes-polish-tourist-attraction/
Signs Your Cat Isn’t Happy
http://catdailynews.com/2023/02/signs-your-cat-isnt-happy/
Tips for Trimming a Cat’s Nails
http://catdailynews.com/2023/02/tips-for-trimming-a-cats-nails/
Landlord Considers a Cat Instead of a Pest Control Service
http://catdailynews.com/2023/02/landlord-considers-a-cat-instead-of-a-pest-control-service/
Cat Rescued From Turkish Earthquake Won’t Leave Rescuer
http://catdailynews.com/2023/02/cat-rescued-from-turkish-earthquake-wont-leave-rescuer/
San Diego Zoo Shares Unusual Cat Facts
http://catdailynews.com/2023/02/san-diego-zoo-shares-unusual-cat-facts/
Cat Overjoyed to Find Lost Toys
http://catdailynews.com/2023/02/cat-overjoyed-to-find-lost-toys/

Cat Comes Home and Embraces Child’s Drawing of the Cat
http://catdailynews.com/2022/12/cat-comes-home-and-embraces-childs-drawing-of-the-cat/
Cat Image in Nazca, Peru Desert
http://catdailynews.com/2022/12/cat-image-in-nazca-peru-desert/
How Cats See the World
http://catdailynews.com/2022/12/how-cats-see-the-world/
Understanding Feral Cats
http://catdailynews.com/2022/12/understanding-feral-cats/
Massive Maine Coon Cat Shocks the Internet
http://catdailynews.com/2022/12/massive-maine-coon-cat-shocks-the-internet/
Cat’s Strict Bedtime Routine
http://catdailynews.com/2022/12/cats-strict-bedtime-routine/
Cat Owner Finds Unique Way to Protect Christmas Tree
http://catdailynews.com/2022/12/cat-owner-finds-unique-way-to-protect-christmas-tree/
Can You Find the Black Cat?
http://catdailynews.com/2022/12/can-you-find-the-black-cat/
Unfriendly Cat Becomes Lovable Pet
http://catdailynews.com/2022/12/unfriendly-cat-becomes-lovable-pet/
Cats May Have Started Bonding With Humans 10,000 Years Ago
ht

Visually Impaired Ct Wakes Up Hearing Impaired Cat
http://catdailynews.com/2022/10/visually-impaired-ct-wakes-up-hearing-impaired-cat/
Cat Plays Dead to Fool Dog
http://catdailynews.com/2022/10/cat-plays-dead-to-fool-dog/
Larry the Cat Chases Away Fox
http://catdailynews.com/2022/10/larry-the-cat-chases-away-fox/
Woman Bakes Cake to Look Like a Litter Box
http://catdailynews.com/2022/10/woman-bakes-cake-to-look-like-a-litter-box/
Cat Enjoys Tormenting Man Who’s Allergic to Cats
http://catdailynews.com/2022/10/cat-enjoys-tormenting-man-whos-allergic-to-cats/
Cat Shocked That Man Knows Cat Language
http://catdailynews.com/2022/10/cat-shocked-that-man-knows-cat-language/
Man Cradles Cat Like a Baby
http://catdailynews.com/2022/10/man-cradles-cat-like-a-baby/
Cat Violently Attacks a Statue
http://catdailynews.com/2022/10/cat-violently-attacks-a-statue/
Cat Annoyed by Visitor
http://catdailynews.com/2022/10/cat-annoyed-by-visitor/
Black Cat Awareness Month
http://catdailynews.com/2022/10/bl

Cats Are Good For Your Health
http://catdailynews.com/2022/08/cats-are-good-for-your-health-5/
Where to Pet Your Cat
http://catdailynews.com/2022/08/where-to-pet-your-cat-2/
Cat Enjoys Stealing Children’s Toys
http://catdailynews.com/2022/08/cat-enjoys-stealing-childrens-toys/
Kitten Freaks Out Over Dog
http://catdailynews.com/2022/08/kitten-freaks-out-over-dog/
Cat Protects Against Rabid Bat
http://catdailynews.com/2022/08/cat-protects-against-rabid-bat/
Paris Cat Hotel Booked Solid
http://catdailynews.com/2022/08/paris-cat-hotel-booked-solid/
Cat Waits For Dead Owner
http://catdailynews.com/2022/08/cat-waits-for-dead-owner/
The Different Traits of Dog and Cat Owners
http://catdailynews.com/2022/08/the-different-traits-of-dog-and-cat-owners/
Brushing a Cat’s Teeth
http://catdailynews.com/2022/08/brushing-a-cats-teeth/
Cat Begs Owner to Turn on iPad Game
http://catdailynews.com/2022/08/cat-begs-owner-to-turn-on-ipad-game/
http://catdailynews.com/page/35/
“Two-Headed Cat” Up For Adoptio

...

Funny Tweets About Cats
http://catdailynews.com/2020/11/funny-tweets-about-cats/
Famous Cats
http://catdailynews.com/2020/11/famous-cats/
Hundreds of Cats and Dogs Flown Out of Hawaii
http://catdailynews.com/2020/11/hundreds-of-cats-and-dogs-flown-out-of-hawaii/
Bringing a Cat to a Hotel
http://catdailynews.com/2020/11/bringing-a-cat-to-a-hotel/
Cats Help Americans Get Through Lockdown
http://catdailynews.com/2020/11/cats-help-americans-get-through-lockdown/
Scaffolding Saves a Cat
http://catdailynews.com/2020/11/scaffolding-saves-a-cat/
Gifts For Cat Lovers
http://catdailynews.com/2020/10/gifts-for-cat-lovers-3/
National Cat Day
http://catdailynews.com/2020/10/national-cat-day-2/
How a Cat Made Aviation History
http://catdailynews.com/2020/10/how-a-cat-made-aviation-history/
Cat Goes Crab Fishing with a Horse
http://catdailynews.com/2020/10/cat-goes-crab-fishing-with-a-horse/
http://catdailynews.com/page/158/
Why Cats Stick Their Butts In Your Face
http://catdailynews.com/2020/10/why-

Library Cats During the Pandemic
http://catdailynews.com/2020/09/library-cats-during-the-pandemic/
Why Cats Rub Their Faces on You
http://catdailynews.com/2020/09/why-cats-rub-their-faces-on-you/
Can You Find the Cat in This Picture?
http://catdailynews.com/2020/09/can-you-find-the-cat-in-this-picture/
The Power of Purr
http://catdailynews.com/2020/09/the-power-of-purr/
The Long Cat Internet Cat Celebrity Passes Away
http://catdailynews.com/2020/09/the-long-cat-internet-cat-celebrity-passes-away/
The Cat in “The Godfather”
http://catdailynews.com/2020/09/the-cat-in-the-godfather/
Cat Sleeps in Piano for a Massage
http://catdailynews.com/2020/09/cat-sleeps-in-piano-for-a-massage/
Cat Therapy Teams
http://catdailynews.com/2020/09/cat-therapy-teams/
Did Curiosity Really Kill the Cat?
http://catdailynews.com/2020/09/did-curiosity-really-kill-the-cat/
Vote For America’s Favorite Pet
http://catdailynews.com/2020/09/vote-for-americas-favorite-pet/
http://catdailynews.com/page/167/
Woman Gets 

Cats May Be Aliens Spying on Us
http://catdailynews.com/2020/08/cats-may-be-aliens-spying-on-us/
Direct-to-Consumer Cat Food Startup
http://catdailynews.com/2020/08/direct-to-consumer-cat-food-startup/
Vote For the Three-Legged Cat
http://catdailynews.com/2020/08/vote-for-the-three-legged-cat/
Cats Help Owners Feel Less Lonely During Pandemic
http://catdailynews.com/2020/08/cats-help-owners-feel-less-lonely-during-pandemic/
Naming a Black Cat
http://catdailynews.com/2020/08/naming-a-black-cat/
Why Cucumbers Terrify Cats
http://catdailynews.com/2020/08/why-cucumbers-terrify-cats/
Flying with a Cat
http://catdailynews.com/2020/08/flying-with-a-cat-3/
Drug-Smuggling Cat Escapes
http://catdailynews.com/2020/08/drug-smuggling-cat-escapes/
Cat Found 30 Miles From Home
http://catdailynews.com/2020/08/cat-found-30-miles-from-home/
Fishing Boat Fishes Out a Cat
http://catdailynews.com/2020/08/fishing-boat-fishes-out-a-cat/
http://catdailynews.com/page/176/
Studying the Behavior of Cats
http://c

Cats in the Old West
http://catdailynews.com/2020/06/cats-in-the-old-west/
The Big Business of Cat Lovers
http://catdailynews.com/2020/06/the-big-business-of-cat-lovers/
Sisters Build Tunnel for Cat
http://catdailynews.com/2020/06/sisters-build-tunnel-for-cat/
Cardboard Cat Bed Shaped Like a Cat
http://catdailynews.com/2020/06/cardboard-cat-bed-shaped-like-a-cat/
Adopting a Cat
http://catdailynews.com/2020/06/adopting-a-cat-2/
Cat TikTok Accounts
http://catdailynews.com/2020/06/cat-tiktok-accounts/
Lions Enjoying Human-Free Park
http://catdailynews.com/2020/06/lions-enjoying-human-free-park/
Women Less Attracted to Men With Cats
http://catdailynews.com/2020/06/women-less-attracted-to-men-with-cats/
Cat Ghost Caught on Camera?
http://catdailynews.com/2020/06/cat-ghost-caught-on-camera/
Cat Reunited with Quarantined Owner
http://catdailynews.com/2020/06/cat-reunited-with-quarantined-owner/
http://catdailynews.com/page/185/
Cat Uses Dogs as a Bed
http://catdailynews.com/2020/06/cat-uses-d

Fostering a Cat
http://catdailynews.com/2020/05/fostering-a-cat/
The Quarantine Cat Film Festival
http://catdailynews.com/2020/05/the-quarantine-cat-film-festival/
Library Cat Dresses Up to Promote Reading
http://catdailynews.com/2020/05/library-cat-dresses-up-to-promote-reading/
Cat Duo Waiting to Be Adopted
http://catdailynews.com/2020/05/cat-duo-waiting-to-be-adopted/
Mick Jagger Holds a Cat
http://catdailynews.com/2020/05/mick-jagger-holds-a-cat/
World’s Smallest Cats
http://catdailynews.com/2020/05/worlds-smallest-cats/
Cats “Working” From Home
http://catdailynews.com/2020/05/cats-working-from-home/
Adopted Cat Saves Owner’s Life
http://catdailynews.com/2020/05/adopted-cat-saves-owners-life/
Taylor Swift’s Cat Worth More Than the Kardashians
http://catdailynews.com/2020/05/taylor-swifts-cat-worth-more-than-the-kardashians/
“Cat in the Wall” Movie
http://catdailynews.com/2020/05/cat-in-the-wall-movie-2/
http://catdailynews.com/page/194/
Taming a Cat in Minecraft
http://catdailynews

In [95]:
print(str(len(news_data)) + ' news links were extracted from catnews webpages.')

1990 news links were extracted from catnews webpages.


### Get  text, category and date from each news link page and write news data to the csv file

In [96]:
n = 0

for idx, i in enumerate(news_data):
    n +=1
    print(n)
    
    protocol_list = requests.get(i[1]).text
    soup = BeautifulSoup(protocol_list, 'lxml')
    
    text_parts = soup.find_all('p')
    text = ''
    for p in text_parts:
        text += p.text
    
    for divi in soup.find_all('div'):
        if divi.get("class") == ['pull-left']:
            date = divi.text.split('  ')[0][:-2]
                
    categories = []
    for divi in soup.find_all('div'):
        if divi.get("class") == ['category-tag']:
            all_a = divi.find_all('a')
            for a in all_a:
                if a.text[0].isupper():
                    categories.append(a.text)
            categories = ', '.join(categories)
            
    with open('news200.csv', 'a', newline='', encoding="utf-8") as csvfile:
        spamwriter = csv.writer(csvfile, delimiter='|')
        spamwriter.writerow([i[0], i[1], date, text, categories])
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990


## 2. Topic classification

### Import cat news data

In [2]:
data = pd.read_csv('news200.csv', sep='|', names=['title', 'link', 'date', 'text', 'category'])
data.head()

,title,link,date,text,category
0,Indiana Jones and a Cat,http://catdailynews.com/2023/06/indiana-jones-...,"June 30, 2023",With the latest Indiana Jones movie out in the...,"Humor, Video"
1,Interesting Cat Facts,http://catdailynews.com/2023/06/interesting-ca...,"June 29, 2023",Many people love cats. However what makes cat...,Humor
2,Cat Gets “Jailed” For Being Too Daring,http://catdailynews.com/2023/06/cat-gets-jaile...,"June 28, 2023",Alice likes traveling in a van on the road wit...,"Humor, Travel"
3,Toys to Keep Your Cat Happy,http://catdailynews.com/2023/06/toys-to-keep-y...,"June 27, 2023","Cats need lots of attention, except when they ...",Humor
4,The Evolution of Cats,http://catdailynews.com/2023/06/the-evolution-...,"June 26, 2023",People love cats so it’s no surprise that peop...,"Science, Egypt"


In [3]:
print("Number of news: ", data.shape[0])

Number of news:  1990


### 1) Preproccessing and cleaning of news data

In [4]:
def remove_stopwords(text):
    return " ".join([w if w not in eng_stopwords else ' ' for w in text.split(' ')])

def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

def lemmatize_text(text):
    return " ".join([lemma.lemmatize(word.lower()) for word in text.split(' ')])

lemma = WordNetLemmatizer()

eng_stopwords = stopwords.words("english")
eng_stopwords.extend(['…', '«', '»', '...'])

In [5]:
data['cleaned_text'] = data['text'].apply(lambda text: lemmatize_text((remove_multiple_spaces(remove_numbers(remove_stopwords(remove_punctuation(text)))))))
data['cleaned_text']

0       with latest indiana jones movie theater people...
1       many people love cat however make cat even lov...
2       alice like traveling van road cat unfortunatel...
3       cat need lot attention except want left alone ...
4       people love cat it’s surprise people also stud...
                              ...                        
1985    even hollywood celebrity stay inside prevent s...
1986    with people unable watch live sporting event c...
1987    if you’ve ever taken picture cat may startled ...
1988    erin merryn daughter ginger tabby named carrot...
1989    in upton merseyside england employee sainsbury...
Name: cleaned_text, Length: 1990, dtype: object

Print unique category sets

In [6]:
print(data['category'].unique(), len(data['category'].unique()))

['Humor, Video' 'Humor' 'Humor, Travel' 'Science, Egypt'
 'Games, Humor, Technology' 'Humor, England' 'Humor, Technology'
 'Homeless cat' 'Humor, Science' 'Games, Technology' 'Games, Humor'
 'Science' 'Humor, Show business' 'Humor, Military' 'Humor, Iceland'
 'Humor, Japan' 'Technology' 'Humor, Morocco' 'Health' 'Uncategorized'
 'Humor, Travel, Thailand' 'Health, Medical' 'Humor, Turkey'
 'Homeless cat, Japan' 'Humor, Travel, Poland' 'Homeless cat, Turkey'
 'Homeless cat, Humor' 'Humor, India' 'Humor, Thailand'
 'Humor, Netherlands, Ukraine' 'Humor, Singapore' 'Health, England'
 'Humor, Travel, Japan' 'Humor, Vietnam' 'Homeless cat, Video'
 'Humor, Canada' 'Homeless cat, Humor, Show business' 'Humor, Australia'
 'Humor, Peru' 'Humor, Ukraine' 'Humor, Sports, Brazil'
 'Homeless cat, Humor, Video' 'Humor, Italy' 'Health, Humor'
 'Health, Science' 'Medical' 'Health, Homeless cat'
 'Humor, Video, England' 'Humor, Greece' 'Crazy cat people, Humor'
 'Homeless cat, Ukraine' 'Financial' 'Show 

As you can see articles can have more than one category. 

Furthermore, country name can also be the last category in category set of article. In this project I will not use counry as a category, that is why I remove all country names from category sets of cat news:

In [7]:
countries = ['England', 'Vietnam', 'Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']

def remove_country_category(category_set):
    categories = category_set.split(', ')
    categories = [category for category in categories if category not in countries]
    categories = ', '.join(categories)
    return categories

data['categories_without_countries'] = data['category'].apply(lambda category_set: remove_country_category(category_set))

### 2) Formation of topic classification system stages

Now let's have a look at "Humor" category. We can see that this category can be seen in the most of news:

In [8]:
print(str(len([c_set for c_set in data['categories_without_countries'] if "Humor" in c_set])) + ' out of ' + str(len(data)) + ' news have "Humor" category in their category set.')

1515 out of 1990 news have "Humor" category in their category set.


Number of all possible category sets:

In [9]:
data['categories_without_countries'].value_counts()[:20]

Humor                                     754
Crazy cat people, Humor                   145
Health                                    123
Humor, Show business                       93
Humor, Video                               82
Homeless cat                               72
Humor, Technology                          62
Crazy cat people, Humor, Show business     47
Humor, Science                             45
Crazy cat people                           39
Health, Medical                            36
Homeless cat, Humor                        31
Science                                    29
Health, Humor                              26
Humor, Travel                              23
Crazy cat people, Health                   19
Crazy cat people, Humor, Technology        15
Games, Humor, Technology                   15
Crazy cat people, Homeless cat             15
Crazy cat people, Humor, Video             13
Name: categories_without_countries, dtype: int64

From the above cell we can see that the most popular category set consists only from category "Humor". After "Humor"we can see category sets that contain "Humor" plus some other category. 

I will reduce the amount of categories that i will try to predict. I take for classification task the following categories: "Humor", "Crazy cat people", "Health", "Video", "Homeless cat", "Show business", "Technology", "Science". All other categories i add to the new category "No category":

In [10]:
def reduce_dataset(category_set):
    new_categories = []
    categories = category_set.replace(" ", "").split(',')
#     print(categories)
    for category in categories:
        if category == 'Humor':
            new_categories.append(category)
            break
    for category in categories:
        if category in ['Crazy cat people', 'Health', 'Video', 'Homeless cat', 'Show business', 'Technology', 'Science']:
            new_categories.append(category)
            break
    if not new_categories:
        new_categories.append('No category')
    categories = ', '.join(new_categories)
    return categories



In [11]:
dataset = data.copy()

In [12]:
dataset['cleaned_category_sets'] = dataset['categories_without_countries'].apply(lambda category_set: reduce_dataset(category_set))

In [13]:
dataset

,title,link,date,text,category,cleaned_text,categories_without_countries,cleaned_category_sets
0,Indiana Jones and a Cat,http://catdailynews.com/2023/06/indiana-jones-...,"June 30, 2023",With the latest Indiana Jones movie out in the...,"Humor, Video",with latest indiana jones movie theater people...,"Humor, Video","Humor, Video"
1,Interesting Cat Facts,http://catdailynews.com/2023/06/interesting-ca...,"June 29, 2023",Many people love cats. However what makes cat...,Humor,many people love cat however make cat even lov...,Humor,Humor
2,Cat Gets “Jailed” For Being Too Daring,http://catdailynews.com/2023/06/cat-gets-jaile...,"June 28, 2023",Alice likes traveling in a van on the road wit...,"Humor, Travel",alice like traveling van road cat unfortunatel...,"Humor, Travel",Humor
3,Toys to Keep Your Cat Happy,http://catdailynews.com/2023/06/toys-to-keep-y...,"June 27, 2023","Cats need lots of attention, except when they ...",Humor,cat need lot attention except want left alone ...,Humor,Humor
4,The Evolution of Cats,http://catdailynews.com/2023/06/the-evolution-...,"June 26, 2023",People love cats so it’s no surprise that peop...,"Science, Egypt",people love cat it’s surprise people also stud...,Science,Science
...,...,...,...,...,...,...,...,...
1985,Jennifer Garner Takes Cat Out in a Stroller,http://catdailynews.com/2020/04/jennifer-garne...,"April 3, 2020",Even Hollywood celebrities have to stay inside...,"Crazy cat people, Humor, Show business",even hollywood celebrity stay inside prevent s...,"Crazy cat people, Humor, Show business",Humor
1986,Cat Goalkeeper,http://catdailynews.com/2020/04/cat-goalkeeper/,"April 3, 2020",With people unable to watch live sporting even...,"Crazy cat people, Humor, Sports, Video",with people unable watch live sporting event c...,"Crazy cat people, Humor, Sports, Video","Humor, Video"
1987,The Glowing Eye Secret of Cat Pictures,http://catdailynews.com/2020/04/the-glowing-ey...,"April 2, 2020","If you’ve ever taken a picture of your cat, yo...",Science,if you’ve ever taken picture cat may startled ...,Science,Science
1988,Ellen DeGeneres Likes Carrot the Cat,http://catdailynews.com/2020/04/ellen-degenere...,"April 2, 2020",Erin Merryn and her daughters have a ginger ta...,"Crazy cat people, Humor, Show business",erin merryn daughter ginger tabby named carrot...,"Crazy cat people, Humor, Show business",Humor




Cat content is funny so it is not suprising that there are so many humorous articles in the dataset. I decided to devide my classification system for two steps:

1. Classify news on humorous or not humorous
2. Classify news on remaining categories
3. Then I will combine results from both classifications and evaluate them.

I will save some data for testing of the final two stages system (humor classification and other categories classification) proportionally. 

In [14]:
threshold_humor = len(dataset[dataset['cleaned_category_sets'] == 'Humor']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor']) * 0.25)
final_test_dataset_humor = dataset[dataset['cleaned_category_sets'] == 'Humor'][threshold_humor:]
working_dataset_humor = dataset[dataset['cleaned_category_sets'] == 'Humor'][:threshold_humor]

threshold_health = len(dataset[dataset['cleaned_category_sets'] == 'Health']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Health']) * 0.25)
final_test_dataset_health = dataset[dataset['cleaned_category_sets'] == 'Health'][threshold_health:]
working_dataset_health = dataset[dataset['cleaned_category_sets'] == 'Health'][:threshold_health]

threshold_no_category = len(dataset[dataset['cleaned_category_sets'] == 'No category']) - round(len(dataset[dataset['cleaned_category_sets'] == 'No category']) * 0.25)
final_test_dataset_no_category = dataset[dataset['cleaned_category_sets'] == 'No category'][threshold_no_category:]
working_dataset_no_category = dataset[dataset['cleaned_category_sets'] == 'No category'][:threshold_no_category]

threshold_humor_video = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Video']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Video']) * 0.25)
final_test_dataset_humor_video = dataset[dataset['cleaned_category_sets'] == 'Humor, Video'][threshold_humor_video:]
working_dataset_humor_video = dataset[dataset['cleaned_category_sets'] == 'Humor, Video'][:threshold_humor_video]

threshold_humor_technology = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Technology']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Technology']) * 0.25)
final_test_dataset_humor_technology = dataset[dataset['cleaned_category_sets'] == 'Humor, Technology'][threshold_humor_technology:]
working_dataset_humor_technology = dataset[dataset['cleaned_category_sets'] == 'Humor, Technology'][:threshold_humor_technology]

threshold_humor_science = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Science']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Science']) * 0.25)
final_test_dataset_humor_science = dataset[dataset['cleaned_category_sets'] == 'Humor, Science'][threshold_humor_science:]
working_dataset_humor_science = dataset[dataset['cleaned_category_sets'] == 'Humor, Science'][:threshold_humor_science]

threshold_humor_health = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Health']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Health']) * 0.25)
final_test_dataset_humor_health = dataset[dataset['cleaned_category_sets'] == 'Humor, Health'][threshold_humor_health:]
working_dataset_humor_health = dataset[dataset['cleaned_category_sets'] == 'Humor, Health'][:threshold_humor_health]

threshold_science = len(dataset[dataset['cleaned_category_sets'] == 'Science']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Science']) * 0.25)
final_test_dataset_science = dataset[dataset['cleaned_category_sets'] == 'Science'][threshold_science:]
working_dataset_science = dataset[dataset['cleaned_category_sets'] == 'Science'][:threshold_science]

threshold_technology = len(dataset[dataset['cleaned_category_sets'] == 'Technology']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Technology']) * 0.25)
final_test_dataset_technology = dataset[dataset['cleaned_category_sets'] == 'Technology'][threshold_technology:]
working_dataset_technology = dataset[dataset['cleaned_category_sets'] == 'Technology'][:threshold_technology]

threshold_video = len(dataset[dataset['cleaned_category_sets'] == 'Video']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Video']) * 0.25)
final_test_dataset_video = dataset[dataset['cleaned_category_sets'] == 'Video'][threshold_video:]
working_dataset_video = dataset[dataset['cleaned_category_sets'] == 'Video'][:threshold_video]

final_test_frames = [final_test_dataset_humor, final_test_dataset_health, final_test_dataset_no_category, 
                    final_test_dataset_humor_video, final_test_dataset_humor_technology, 
                    final_test_dataset_humor_science, final_test_dataset_humor_health,
                    final_test_dataset_science, final_test_dataset_technology, final_test_dataset_video]
final_test_dataset = pd.concat(final_test_frames)


working_dataset = [working_dataset_humor, working_dataset_health, working_dataset_no_category, 
                    working_dataset_humor_video, working_dataset_humor_technology, 
                    working_dataset_humor_science, working_dataset_humor_health,
                    working_dataset_science, working_dataset_technology, working_dataset_video]
working_dataset = pd.concat(working_dataset)

I will continue working with this dataset:

In [15]:
working_dataset['cleaned_category_sets'].value_counts()

Humor                893
Health               161
No category          146
Humor, Video          84
Humor, Technology     79
Humor, Science        43
Humor, Health         38
Science               31
Technology            15
Video                  3
Name: cleaned_category_sets, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(working_dataset['cleaned_text'], working_dataset['cleaned_category_sets'], test_size=0.2, random_state=42)

### 2) Classification for humorous or not humorous articles

### Features engineering

For classification for humorous or not humorous articles I make labels "Humor" or "not Humor" according each article category set.

In [17]:
def remove_categories_exept_humor(category_set):
    if "Humor" in category_set:
        return "Humor"
    else:
        return "not Humor"

In [18]:
working_dataset['category_humor_task'] = working_dataset['cleaned_category_sets'].apply(lambda category_set: remove_categories_exept_humor(category_set))

In [19]:
print('Number of humorous news: ', len(working_dataset[working_dataset['category_humor_task'] == "Humor"]))
print('Number of not humorous news: ', len(working_dataset[working_dataset['category_humor_task'] == "not Humor"]))

Number of humorous news:  1137
Number of not humorous news:  356


I want to make number of samples for each class similar for training

In [20]:
humor_train_dataset = working_dataset[working_dataset['category_humor_task'] == "Humor"][:500]
humor_test_dataset = working_dataset[working_dataset['category_humor_task'] == "Humor"][500:]
humor_train_dataset = pd.concat([humor_train_dataset, working_dataset[working_dataset['category_humor_task'] == "not Humor"]])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(humor_train_dataset['cleaned_text'], humor_train_dataset['category_humor_task'], test_size=0.2, random_state=42)

### Choose the right model

Let's look at performance of two models: SGDClassifier (linear models with stochastic gradient descent) and KNeighborsClassifier (k-nearest neighbors):

In [22]:
sgd_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])

knb_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

sgd_clf.fit(X_train, y_train)
knb_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

#### SGDClassifier results

In [23]:
predicted_sgd = sgd_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

       Humor       0.80      0.74      0.77       104
   not Humor       0.64      0.72      0.68        68

    accuracy                           0.73       172
   macro avg       0.72      0.73      0.73       172
weighted avg       0.74      0.73      0.73       172



#### KNeighborsClassifier

In [24]:
predicted_knb = knb_clf.predict(X_test)
print(metrics.classification_report(predicted_knb, y_test))

              precision    recall  f1-score   support

       Humor       0.85      0.69      0.77       118
   not Humor       0.53      0.74      0.62        54

    accuracy                           0.71       172
   macro avg       0.69      0.72      0.69       172
weighted avg       0.75      0.71      0.72       172



SGDClassifier has better results than KNeighborsClassifier, that's why I will use SGDClassifier for the further work.

### Hyperparameters optimization

Next I do hyperparameters optimization for SGDClassifier using 5-fold cross-validation:

In [25]:
sgd_clf = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])

parameters = {
    'sgd_clf__loss': ['hinge', 'modified_huber'], 
    'sgd_clf__penalty': ['l2', 'l1', 'elasticnet']
}

clf = GridSearchCV(sgd_clf, parameters, n_jobs=5)
clf.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('sgd_clf',
                                        SGDClassifier(random_state=42))]),
             n_jobs=5,
             param_grid={'sgd_clf__loss': ['hinge', 'modified_huber'],
                         'sgd_clf__penalty': ['l2', 'l1', 'elasticnet']})

In [26]:
print("Best parameter (CV score=%0.3f):" % clf.best_score_)
print(clf.best_params_)

Best parameter (CV score=0.753):
{'sgd_clf__loss': 'hinge', 'sgd_clf__penalty': 'l2'}


### Humorous articles classification results

I train our SGDClassifier model again with the best hyperparameters:

In [27]:
humor_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42, loss='hinge', penalty='l2'))])


humor_clf.fit(X_train, y_train)

predicted_humorous_clf = humor_clf.predict(X_test)
print(metrics.classification_report(predicted_humorous_clf, y_test))


              precision    recall  f1-score   support

       Humor       0.80      0.74      0.77       104
   not Humor       0.64      0.72      0.68        68

    accuracy                           0.73       172
   macro avg       0.72      0.73      0.73       172
weighted avg       0.74      0.73      0.73       172



Save model as pickle file:

In [50]:
with open("humor_clf.pkl", "wb") as file: 
    pickle.dump(humor_clf, file) 

The F1 score for predicting humorous news about cats was 0.77, while for predicting non-humorous news about cats it was 0.68. Average F1-score is 0.73.

### 3) Classification for other categories

### Features engineering

I replace "Humor" category from each article set with "No category". So now the task is to predict all categories exept "Humor" plus "No category" category.

In [31]:
def remove_humor_category(categories):
    categories = categories.split(', ')
    if 'Humor' in categories and len(categories) > 1:
        categories.remove('Humor')
    elif 'Humor' in categories and len(categories) == 1:
        categories = [i.replace('Humor', 'No category') for i in categories]
    categories = ', '.join(categories)
    return categories


In [32]:
working_dataset['category_other_categ_task'] = working_dataset['cleaned_category_sets'].apply(lambda category_set: remove_humor_category(category_set))

In this dataset we have too many samples that belong to "No category" class. First I decided to reduce their amount to make dataset more balanced, but it just made classification results worse. Then I decided to keep the amount of data from different categories as it is.

In [33]:
working_dataset['category_other_categ_task'].value_counts()

No category    1039
Health          199
Technology       94
Video            87
Science          74
Name: category_other_categ_task, dtype: int64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(working_dataset['cleaned_text'], working_dataset['category_other_categ_task'], test_size=0.2, random_state=23)

### Choose the right model

For this task let's also look at performance of two models: SGDClassifier (linear models with stochastic gradient descent) and KNeighborsClassifier (k-nearest neighbors):

In [35]:
sgd_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])

knb_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

sgd_clf.fit(X_train, y_train)
knb_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

#### SGDClassifier results

In [36]:
predicted_sgd = sgd_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

      Health       0.66      0.66      0.66        41
 No category       0.90      0.85      0.87       223
     Science       0.36      0.38      0.37        13
  Technology       0.62      0.67      0.64        12
       Video       0.21      0.40      0.28        10

    accuracy                           0.78       299
   macro avg       0.55      0.59      0.56       299
weighted avg       0.81      0.78      0.79       299



#### KNeighborsClassifier results

In [37]:
predicted_knb = knb_clf.predict(X_test)
print(metrics.classification_report(predicted_knb, y_test))

              precision    recall  f1-score   support

      Health       0.59      0.71      0.64        34
 No category       0.93      0.80      0.86       246
     Science       0.07      1.00      0.13         1
  Technology       0.54      0.44      0.48        16
       Video       0.05      0.50      0.10         2

    accuracy                           0.77       299
   macro avg       0.44      0.69      0.44       299
weighted avg       0.86      0.77      0.81       299



In this task SGDClassifier also worked better then KNeighborsClassifier. so I take SGDClassifier as a model for other categories classification. Firstly I optimize its hyperparameters.

In [38]:
sgd_clf = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])

parameters = {
    'sgd_clf__loss': ['hinge', 'modified_huber'], 
    'sgd_clf__penalty': ['l2', 'l1', 'elasticnet']
}

clf = GridSearchCV(sgd_clf, parameters, n_jobs=5)
clf.fit(X_train, y_train)

print("Best parameter (CV score=%0.3f):" % clf.best_score_)
print(clf.best_params_)

Best parameter (CV score=0.774):
{'sgd_clf__loss': 'modified_huber', 'sgd_clf__penalty': 'elasticnet'}


In [39]:
other_categ_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42, loss='modified_huber', penalty='elasticnet'))])

other_categ_clf.fit(X_train, y_train)

predicted_other_categ_clf = other_categ_clf.predict(X_test)
print(metrics.classification_report(predicted_other_categ_clf, y_test))

              precision    recall  f1-score   support

      Health       0.61      0.64      0.62        39
 No category       0.90      0.83      0.86       230
     Science       0.14      0.22      0.17         9
  Technology       0.54      0.70      0.61        10
       Video       0.21      0.36      0.27        11

    accuracy                           0.77       299
   macro avg       0.48      0.55      0.51       299
weighted avg       0.80      0.77      0.78       299



Save model to pickle file:

In [40]:
with open("other_categ_clf.pkl", "wb") as file: 
    pickle.dump(other_categ_clf, file) 

Average F1 score is 0.51 for 7 categories classification task. "Science" and "Video" categories  are the worst predicted categories.

### 4) Сombining two classificators together

Now I combine two stages - humor classification and other categories classification - in one system and compare results with true ones. For testing these two stages together I have data, which i saved earlier. These saved data were not used in training and testing of separate models. 

### Prediction and results pre processing

In [41]:
X_test = final_test_dataset['cleaned_text']

predicted_humor_clf = humor_clf.predict(X_test)
predicted_other_categ_clf = other_categ_clf.predict(X_test)

pred_humor = []
for i in predicted_humor_clf:
    pred_humor.append(str(i))

pred_other_categ = []
for i in predicted_other_categ_clf:
    pred_other_categ.append(str(i))

    
combine_results = [a +','+ b for a, b in zip(pred_humor, pred_other_categ)]
combine_results = [str_categ.split(',') for str_categ in combine_results]
combine_results = pd.Series(combine_results)


In [42]:
def remove_additional_categories(categories):
    categ = [i for i in categories if i != 'No category']
    categ = [i for i in categ if i != 'not Humor']
    return ', '.join(categ)
       
combine_results_no_category_removed = combine_results.apply(lambda category: remove_additional_categories(category))

### Comparison of results

In [43]:
number_of_full_true_answers = 0
final_test_dataset['cleaned_category_sets']
for idx_ct, category_true in enumerate(final_test_dataset['cleaned_category_sets']):
    for idx_p, category_pred in enumerate(combine_results_no_category_removed):
        if idx_ct == idx_p:
            if category_true == category_pred:
                number_of_full_true_answers += 1

number_of_partly_true_answers = 0              
final_test_dataset['cleaned_category_sets']
for idx_ct, category_true in enumerate(final_test_dataset['cleaned_category_sets']):
    for idx_p, category_pred in enumerate(combine_results_no_category_removed):
        if idx_ct == idx_p:
            category_true_list = category_true.split(',')
            category_pred_list = category_pred.split(',')
            for categ in category_pred_list:
                if categ in category_true_list:
                    number_of_partly_true_answers += 1
                    break

### Results

In [44]:
print(str(number_of_full_true_answers) + ' of ' + str(len(final_test_dataset)) + ' category sets were predicted completely right')
print(str(number_of_partly_true_answers) + ' of ' + str(len(final_test_dataset)) + ' predicted category sets has at least one right predicted category.')

293 of 497 category sets were predicted completely right
345 of 497 predicted category sets has at least one right predicted category.


In [45]:
print('Accuracy that all categories were predicted correctly is ', str(round(number_of_full_true_answers/len(final_test_dataset)*100,2)))
print('Accuracy that at lest one category was predicted correctly is ', str(round(number_of_partly_true_answers/len(final_test_dataset)*100,2)))

Accuracy that all categories were predicted correctly is  58.95
Accuracy that at lest one category was predicted correctly is  69.42


## 3. Sentiment analysis

For sentiment analysis I use pre-trained VADER tool. 

VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.

I don't train algorithm, I just predict if article is positive, negative or neutral with already pre-tarined tool.

In [46]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#calculate the negative, positive, neutral and compound scores, plus verbal evaluation
def sentiment_vader(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"

    elif sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"

    else :
        overall_sentiment = "Neutral"
  
    return overall_sentiment



In [47]:
data['sentiment'] = data['text'].apply(lambda text: sentiment_vader(text))

In [48]:
data.head()

,title,link,date,text,category,cleaned_text,categories_without_countries,sentiment
0,Indiana Jones and a Cat,http://catdailynews.com/2023/06/indiana-jones-...,"June 30, 2023",With the latest Indiana Jones movie out in the...,"Humor, Video",with latest indiana jones movie theater people...,"Humor, Video",Positive
1,Interesting Cat Facts,http://catdailynews.com/2023/06/interesting-ca...,"June 29, 2023",Many people love cats. However what makes cat...,Humor,many people love cat however make cat even lov...,Humor,Positive
2,Cat Gets “Jailed” For Being Too Daring,http://catdailynews.com/2023/06/cat-gets-jaile...,"June 28, 2023",Alice likes traveling in a van on the road wit...,"Humor, Travel",alice like traveling van road cat unfortunatel...,"Humor, Travel",Positive
3,Toys to Keep Your Cat Happy,http://catdailynews.com/2023/06/toys-to-keep-y...,"June 27, 2023","Cats need lots of attention, except when they ...",Humor,cat need lot attention except want left alone ...,Humor,Positive
4,The Evolution of Cats,http://catdailynews.com/2023/06/the-evolution-...,"June 26, 2023",People love cats so it’s no surprise that peop...,"Science, Egypt",people love cat it’s surprise people also stud...,Science,Positive


In [49]:
print('Number of positive cat news out of all news', str(len(data[data['sentiment'] == "Positive"])) + '/' + str(len(data)))
print('Number of negative cat news out of all news', str(len(data[data['sentiment'] == "Negative"])) + '/' + str(len(data)))
print('Number of neutral cat news out of all news', str(len(data[data['sentiment'] == "Neutral"])) + '/' + str(len(data)))
print('Cat news are mostly positive.')

Number of positive cat news out of all news 1600/1990
Number of negative cat news out of all news 364/1990
Number of neutral cat news out of all news 26/1990
Cat news are mostly positive.
